In [1]:
import re
import csv
import sys, random
import pandas as pd
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW

###   Random Sampling을 통한 new train dataset 구성

In [ ]:
def random_positive(path,wr,duplicate_check):
    numbers = list(range(1,301))
    random_num = random.choice(numbers)
    random_dir = "problem"+str(random_num).zfill(3)
    code = []
    tuple_set = []
    for _ in range(2):
        random_file = random.randint(1,150)
        num = 0
        while (random_num,random_file) in duplicate_check and len(numbers)>0:
            random_file = random.randint(1,150)
            num += 1
            if num > 100:
                numbers.remove(random_num)
                random_num = random.choice(numbers)
                num=0
        tuple_set.append((random_num,random_file))
        file_name = path+"/"+random_dir+"/"+random_dir+"_"+str(random_file)+".py"

        with open(file_name,'r') as file:
            code.append("".join(file.readlines()))
    wr.writerow([code[0],code[1],1])
    return tuple_set


def random_negative(path,wr,duplicate_check):
    code = []
    tuple_set = []
    random_num1 = random.randint(1,300)
    random_file1 = random.randint(1,150)
    num1 = 0
    while (random_num1,random_file1) in duplicate_check:
        random_file1 = random.randint(1,150)
        num1 += 1
        if num1>100:
            random_num1 = random.randint(1,300)
            num1=0
    tuple_set.append((random_num1,random_file1))
    
    while(True):
        random_num2 = random.randint(1,300)
        if random_num2 != random_num1:
            break
    random_file2 = random.randint(1,150)
    num2=0
    while (random_num2,random_file2) in duplicate_check:
            random_file2 = random.randint(1,150)
            num2 +=1
            if num2>100:
                while(True):
                    random_num2 = random.randint(1,300)
                    if random_num2 != random_num1:
                        break
    tuple_set.append((random_num2,random_file2))
    
    random_dir1 = "problem"+str(random_num1).zfill(3)
    random_dir2 = "problem"+str(random_num2).zfill(3)
    
    file_name1 = path+"/"+random_dir1+"/"+random_dir1+"_"+str(random_file1)+".py"
    file_name2 = path+"/"+random_dir2+"/"+random_dir2+"_"+str(random_file2)+".py"
    with open(file_name1,'r') as file:
        code.append("".join(file.readlines()))
    with open(file_name2,'r') as file:
        code.append("".join(file.readlines()))
    
    wr.writerow([code[0],code[1],0])
    return tuple_set


if __name__ == "__main__":
    path = str(sys.path[0]) + "/code/"
    f = open('new_train_sample_5.csv','w',newline='\n') 
    wr = csv.writer(f)
    wr.writerow(['code1','code2','similar'])
    row = 0
    random_list = [random.randint(0,1) for r in range(45088)]
    duplicate_check_positive = []
    duplicate_check_negative = []

    while(row<45088): #pair 개수
        if row%100==0: print(row)
        if random_list[row] == 0:
            duplicate_check_positive += random_positive(path,wr,duplicate_check_positive)
        elif random_list[row] == 1:
            duplicate_check_negative += random_negative(path,wr,duplicate_check_negative)
        row +=1

    f.close()

In [3]:
import csv

# 두 파이썬 코드 파일 경로
file_path1 = "baekjoon68367943(1).py"
file_path2 = "baekjoon68367943(2).py"
is_similar = 1  # 유사한 코드인 경우 1, 유사하지 않은 경우 0

# 첫 번째 파일 읽기
with open(file_path1, 'r') as file1:
    code1 = file1.read()

# 두 번째 파일 읽기
with open(file_path2, 'r') as file2:
    code2 = file2.read()

# CSV 파일 열기
with open('new_train_sample_5.csv', 'a', newline='') as file:
    writer = csv.writer(file)

    # 샘플 추가
    writer.writerow([code1, code2, is_similar])

print("샘플이 CSV 파일에 추가되었습니다.")

샘플이 CSV 파일에 추가되었습니다.


### 모델 구현 및 학습, 추론


*   Load Train dataset

In [5]:
train = pd.read_csv('new_train_sample_5.csv', encoding = 'utf-8')
df = pd.DataFrame(train)
df.tail(5)

,code1,code2,similar
44787,"n,m = map(int,input().split())\nh = list(map(i...","N, M = map(int, input().split())\nH = list(map...",1
44788,"n = int(input())\na = list(map(int, input().sp...","n, m = map(int, input().split())\nc = list(map...",0
44789,def main():\n n = int(input())\n a_list ...,"N, M = input().split()\nN = int(N)\nM = int(M)...",0
44790,"def gcd(x,y):\n if x%y!=0:\n return ...",#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,0
44791,# https://www.acmicpc.net/source/68367943\nstr...,# https://www.acmicpc.net/source/68367943\nstr...,1



*   Define Model, tokenizer


In [6]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/longcoder-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/longcoder-base")

batch_size = 16
epoch_num = 16
MAX_LEN = 512
learning_rate = 2e-4

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* Preprocessing



In [7]:
def preprocess(df,file_name):
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [8]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/44792 [00:00<?, ? examples/s]

* 데이터 나누기

In [9]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

* optimizers 설정

In [10]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_932432/4263078525.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
gc.collect()
torch.cuda.empty_cache()

* 학습

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

# Model Train 및 Evaluation 종료.

In [ ]:
TEST = "test.csv"
SAMPLE = "sample_submission.csv"

test = pd.read_csv('test.csv', encoding = 'utf-8')
df_test = pd.DataFrame(test)

preprocess(df_test,"preprocess_test.csv")

test_dataset = load_dataset("csv",data_files = "preprocess_test.csv")['train']
test_dataset = test_dataset.map(tokenized,remove_columns=['code1','code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SAMPLE)
df['similar'] = np.argmax(predictions.predictions,axis=-1)
df.to_csv('./submission.csv',index=False)

# Model save

In [89]:
model.save_pretrained("code_similarity_model")
tokenizer.save_pretrained("code_similarity_model")

('code_similarity_model/tokenizer_config.json',
 'code_similarity_model/special_tokens_map.json',
 'code_similarity_model/vocab.json',
 'code_similarity_model/merges.txt',
 'code_similarity_model/added_tokens.json',
 'code_similarity_model/tokenizer.json')

# Model 불러온 후 적용

In [90]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 불러온 모델과 토크나이저의 경로
model_path = "code_similarity_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Python 코드가 들어있는 두 개의 py 파일 경로
# https://www.acmicpc.net/status?from_mine=1&problem_id=17609&user_id=chan4im
file_path1 = "code3.py"
file_path2 = "code4.py"

# 두 개의 py 파일을 읽어와서 코드를 가져옴
with open(file_path1, 'r') as file1:
    code1 = file1.read()

with open(file_path2, 'r') as file2:
    code2 = file2.read()

# 코드 유사성 평가를 위한 입력 생성
inputs = tokenizer(code1, code2, padding=True, max_length=MAX_LEN, truncation=True, return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits

In [91]:
print(predictions)

tensor([[-2.9283,  2.6643]], grad_fn=<AddmmBackward0>)


In [92]:
import torch
import torch.nn.functional as F

logits = predictions[0]
probabilities = F.softmax(logits, dim=0)
similarity_percentage = probabilities[1].item() * 100

print(f"유사성: {similarity_percentage:.2f}%")


유사성: 99.63%
